<a href="https://colab.research.google.com/github/eisbetterthanpi/python/blob/master/Autoencoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import torch
import torch.nn as nn
# import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

import matplotlib.pyplot as plt
import numpy as np
# https://www.geeksforgeeks.org/implementing-an-autoencoder-in-pytorch/


# transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])
# train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
# test_dataset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_data = torchvision.datasets.MNIST(root="data", train=True, download=True,transform=transforms.ToTensor(),)
test_data = torchvision.datasets.MNIST(root="data", train=False, download=True, transform=transforms.ToTensor(),) #opt no download
batch_size = 4 # 512
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=False)



In [ ]:


class AE(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = torch.nn.Sequential(
            nn.Linear(28 * 28, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 36),
            nn.ReLU(),
            nn.Linear(36, 18),
            nn.ReLU(),
            nn.Linear(18, 10),
        )
          
        self.decoder = torch.nn.Sequential(
            nn.Linear(10, 18),
            nn.ReLU(),
            nn.Linear(18, 36),
            nn.ReLU(),
            nn.Linear(36, 64),
            nn.ReLU(),
            nn.Linear(64, 128),
            nn.ReLU(),
            nn.Linear(128, 28 * 28),
            nn.Sigmoid(),
        )
  
    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

device = "cuda" if torch.cuda.is_available() else "cpu"
model = AE().to(device)
# model = AE()
loss_function = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-1, weight_decay = 1e-8)
# optimizer = torch.optim.Adam(model.parameters(), lr = 1e-3, weight_decay = 1e-8)

In [ ]:

epochs = 20
outputs = []
losses = []
for epoch in range(epochs):
    for (image, _) in train_loader:
        image = image.to(device).reshape(-1, 28*28)
        # image = image.reshape(-1, 28*28)
        reconstructed = model(image)
        loss = loss_function(reconstructed, image)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        losses.append(loss)
    print(sum(losses[-batch_size:])/batch_size)
    outputs.append((epochs, image, reconstructed))
  
# https://blog.paperspace.com/adversarial-autoencoders-with-pytorch/
plt.style.use('fivethirtyeight')
plt.xlabel('Iterations')
plt.ylabel('Loss')
plt.plot(losses[-100:])

tensor(0.0712, grad_fn=<DivBackward0>)
tensor(0.0669, grad_fn=<DivBackward0>)
tensor(0.0814, grad_fn=<DivBackward0>)
tensor(0.0680, grad_fn=<DivBackward0>)
tensor(0.0661, grad_fn=<DivBackward0>)
tensor(0.0648, grad_fn=<DivBackward0>)
tensor(0.0752, grad_fn=<DivBackward0>)
tensor(0.0602, grad_fn=<DivBackward0>)


In [ ]:
for i, item in enumerate(image):
  item = item.reshape(-1, 28, 28)
  plt.imshow(item[0])
  
for i, item in enumerate(reconstructed):
  item = item.reshape(-1, 28, 28)
  plt.imshow(item[0])